# Train image classifier with Huggingface

In [ ]:
from train_fast_rcnn import sample_environment
for batch in sample_environment(batch_size=32, N=100):
    break

In [ ]:
images, targets, raw_images, inv = batch

In [ ]:
from PIL import Image
import numpy as np

img = Image.fromarray((images[0].cpu().numpy()*255).astype(np.uint8).transpose(1, 2, 0))
img.show()

In [ ]:
import torch
# save images
torch.save(images, 'images.pth')

In [ ]:
import torch
from train_fast_rcnn import IntegratedBoundingBoxModel

model = IntegratedBoundingBoxModel()
model.load("/plancraft/plancraft_fast_rcnn.pth")
model.eval()
model = model.cuda()

images = torch.load("images.pth")
with torch.no_grad():
    images = images.cuda()
    predictions = model(images)

In [ ]:
from minerl.herobraine.hero.mc import ALL_ITEMS
from PIL import Image, ImageDraw

for img_idx in range(len(images)):
    # generate image and target for validation
    img = Image.fromarray(raw_images[img_idx].copy())
    for box in targets[img_idx]["boxes"]:
        draw = ImageDraw.Draw(img)

    for box_idx in range(len(predictions[img_idx]["boxes"])):
        box = predictions[img_idx]["boxes"][box_idx]
        score = predictions[img_idx]["scores"][box_idx]
        label = predictions[img_idx]["labels"][box_idx]
        quantity = predictions[img_idx]["quantities"][box_idx]

        if score > 0.05:
            draw = ImageDraw.Draw(img)
            draw.rectangle(box.cpu().tolist(), outline="green")
            label_name = ALL_ITEMS[label.item()]
            draw.text(
                (box[0], box[1]),
                f"{label_name}",
                fill="green",
            )
            draw.text(
                (box[0], box[1] + 10),
                f"{quantity.item()}",
                fill="blue",
            )

    img.show()
    # break